Importing needed libraries for preprocessing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import h5py 
import librosa
import librosa.display
import os, sys 
import time
import pandas as pd
import pickle

Sound feature extraction method: Mel-frequency cepstral coefficients

The time-series feature matrices of one sound extracted by mfcc resulted in a (173, 40) input data per sound.

The column direction contained the time-series data, whereas the row direction contained the feature data of a specific time. Only four-second long files were used, allowing the unification of the input data size.

In [2]:
from google.colab import drive #Can ignore if done locally
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#!tar -xvf  '/content/gdrive/Shared drives/CS1470-Final/UrbanSound8K.tar.gz' -C '/content/gdrive/Shared drives/CS1470-Final'

In [3]:
def get_features(filename):
  num_mfcc = 40
  try:
    audio, SR = librosa.load(filename)
    mfcc = librosa.feature.mfcc(audio, sr=SR, n_mfcc=num_mfcc)
  except Exception as e:
    print('Error encountered getting features, file: ', filename)
    return None

  return mfcc

audio_path = '/content/gdrive/Shared drives/CS1470-Final/UrbanSound8K/audio'
csv_metadata = pd.read_csv(audio_path + '/../metadata/UrbanSound8K.csv')

collected_mfccs = []

for i, row in csv_metadata.iterrows(): 
  curr_file = os.path.join(os.path.abspath(audio_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
  curr_class = row['classID']
  if row['end'] - row['start'] == 4:
    curr_mfcc = np.transpose(get_features(curr_file))
    collected_mfccs.append([curr_mfcc, curr_class])

pickle.dump(collected_mfccs, open('mfccs.pkl', 'wb'))

from google.colab import files
files.download('mfccs.pkl')

with open('mfccs.pkl', 'rb') as f:
  extract = pickle.load(f)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>